In [1]:
!pip install -q transformers==4.36.0
!pip install -q bitsandbytes==0.41.3 accelerate==0.26.0
!pip install -q -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 70.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.2.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.36.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.7/270.7 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 91.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 81.5 MB/s eta

In [2]:
import numpy as np
from PIL import Image
# import uuid
import json
import os
from pprint import pprint
import random
import requests
from io import BytesIO
import pickle
import pandas as pd

In [3]:
import torch
from transformers import AutoProcessor, MllamaForConditionalGeneration
from sentence_transformers import SentenceTransformer
from sklearn.neighbors import NearestNeighbors

In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
from google.colab import userdata
hf = userdata.get('llama')

## Reformatting Images - DO NOT RUN

In [25]:
test_files = '/content/drive/MyDrive/capstone_project/image_data/test_images/'
list_of_categories = os.listdir(test_files)
print(list_of_categories)

['plastic_cup', 'paper_cup', 'metal_beverage_can', 'metal_food_can', 'glass_bottle', 'plastic_jug', 'plastic_bag', 'plastic_bottle', 'plastic_takeout_container', 'styrofoam_cup', 'cardboard_box']


In [23]:
def image_open(file_path):

  im = Image.open(file_path)

  if file_path.endswith(".jpg"):
    img_np = (resize(np.asarray(im), (300, 250, 3))*255).astype('uint8')
  elif file_path.endswith(".png"):
    im_3d = np.asarray(im)[:, :, :3]
    img_np = (resize(np.asarray(im_3d), (300, 250, 3))*255).astype('uint8')

  return img_np

In [27]:
print(test_files + 'shiudf/')

/content/drive/MyDrive/capstone_project/image_data/test_images/shiudf/


In [29]:
y_data = []
x_data = []

for item in list_of_categories:
  for im in os.listdir(test_files + f'{item}/'):
    x_data.append(image_open(test_files + f'{item}/{im}'))
    y_data.append(item)

#np.save('/content/drive/My Drive/capstone_personal/data/test_x_data.npy', x_data)
#np.save('/content/drive/My Drive/capstone_personal/data/test_y_data.npy', y_data)


/content/drive/MyDrive/capstone_project/image_data/test_images/plastic_cup/IMG_8261.jpg
/content/drive/MyDrive/capstone_project/image_data/test_images/paper_cup/IMG_8254.jpg
/content/drive/MyDrive/capstone_project/image_data/test_images/metal_beverage_can/IMG_8255.jpg
/content/drive/MyDrive/capstone_project/image_data/test_images/metal_food_can/IMG_8267.jpg
/content/drive/MyDrive/capstone_project/image_data/test_images/glass_bottle/IMG_8312.jpg
/content/drive/MyDrive/capstone_project/image_data/test_images/plastic_jug/IMG_8357.jpg
/content/drive/MyDrive/capstone_project/image_data/test_images/plastic_bag/IMG_8374.jpg
/content/drive/MyDrive/capstone_project/image_data/test_images/plastic_bottle/IMG_8361.jpg
/content/drive/MyDrive/capstone_project/image_data/test_images/plastic_takeout_container/IMG_8264.jpg
/content/drive/MyDrive/capstone_project/image_data/test_images/styrofoam_cup/IMG_8295.jpg
/content/drive/MyDrive/capstone_project/image_data/test_images/cardboard_box/IMG_8370.jpg


In [ ]:
len(y_data)

84

## test images results

In [31]:
x_data = np.load('/content/drive/MyDrive/capstone_project/image_data/test_x_data.npy', allow_pickle=True)
y_data = np.load('/content/drive/MyDrive/capstone_project/image_data/test_y_data.npy', allow_pickle=True)

In [30]:
## load RAG files

rag_filepath = '/content/drive/MyDrive/capstone_project/RAG_files/'

with open(rag_filepath+'test_instructions.json', 'r') as openfile:
    recycling_instructions_open = json.load(openfile)

with open(rag_filepath + 'test_names_reference_list', 'rb') as dict_file:
    rag_embeddings = pickle.load(dict_file)

with open(rag_filepath + 'locations_idx_list', 'rb') as loc_dict_file:
    locations_json_idx_dict = pickle.load(loc_dict_file)

instruct_mapping_csv = pd.read_csv(rag_filepath + "test_instruction_mapping_df.csv")

In [38]:
print(rag_embeddings)

{'aluminum can': 0, 'aluminum food can': 1, 'tin can': 1, 'steel can': 1, 'canned food container': 1, 'canned food can': 1, 'soda can': 2, 'beverage can': 2, 'soft drink can': 2, 'pepsi can': 2, 'coca cola can': 2, 'paper cup': 3, 'hot beverage cup': 3, 'coffee cup': 3, 'lined paper cup': 3, 'disposable coffee cup': 3, 'white paper cup': 3, 'cardboard': 4, 'cardboard box': 4, 'box': 4, 'packing box': 4, 'cereal box': 4, 'glass bottle': 5, 'glass drink container': 5, 'alcohol bottle': 5, 'beer bottle': 5, 'water bottle': 6, 'disposable water bottle': 6, 'plastic bottle': 6, 'plastic soda bottle': 6, 'plastic cup': 7, 'clear plastic cup': 7, 'iced coffee cup': 7, 'clear plastic bag': 8, 'plastic bag': 8, 'thin plastic': 8, 'plastic jug': 9, 'milk jug': 9, 'plastic takeout container': 10, 'clear plastic clamshell': 10, 'styrofoam cup': 11, 'styrofoam food container': 11, 'styrofoam packing block': 12}


In [32]:
# table created for true values for stage 2
classes_col = np.repeat(np.arange(0, 14), 2)
loc_name = np.tile(['Cook County, IL', 'Palm Desert, CA'], 14)
loc_idx = np.tile([1, 2], 14)
recycling_generic_names = np.repeat(['aluminum can', 'metal food can', 'beverage can', 'disposable cup', 'cardboard box',
                                     'glass bottle', 'plastic bottle', 'plastic cup', 'plastic bag', 'plastic jug', 'plastic takeout container',
                                     'styrofoam food container', 'styrofoam packing block', 'other'], 2)
recyclability = ['yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'no', 'no','yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'no', 'no', 'no', 'no',
                 'yes', 'yes', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'no']

test_s2_answers_df= pd.DataFrame({'class': classes_col, 'location': loc_name, 'location_id': loc_idx, 'generic names': recycling_generic_names, 'recyclability': recyclability})
test_s2_answers_df

,class,location,location_id,generic names,recyclability
0,0,"Cook County, IL",1,aluminum can,yes
1,0,"Palm Desert, CA",2,aluminum can,yes
2,1,"Cook County, IL",1,metal food can,yes
3,1,"Palm Desert, CA",2,metal food can,yes
4,2,"Cook County, IL",1,beverage can,yes
5,2,"Palm Desert, CA",2,beverage can,yes
6,3,"Cook County, IL",1,disposable cup,no
7,3,"Palm Desert, CA",2,disposable cup,no
8,4,"Cook County, IL",1,cardboard box,yes
9,4,"Palm Desert, CA",2,cardboard box,yes


In [36]:
# true class ids for stage 1 - based off of assigned class labels on embeddings words list (see rag files)
class_ref_dict = {'cardboard_box':4,
                  'glass_bottle':5,
                  'metal_beverage_can':2,
                  'metal_food_can':1,
                  'paper_cup':3,
                  'plastic_bag':8,
                  'plastic_bottle':6,
                  'plastic_cup':7,
                  'plastic_jug':9,
                  'plastic_takeout_container':10,
                  'styrofoam_cup':11}

## Modeling

In [6]:
model_id = "meta-llama/Llama-3.2-11B-Vision-Instruct"

model = MllamaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    token=hf
)
#model.tie_weights()

processor = AutoProcessor.from_pretrained(model_id, token=hf)

config.json:   0%|          | 0.00/5.07k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/89.4k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

preprocessor_config.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/5.15k [00:00<?, ?B/s]

In [ ]:
# load encoder model for embedding retrieval

encoder_model = SentenceTransformer('sentence-transformers/LaBSE')

### Testing with 1 image

In [26]:
from torch.cuda.amp import autocast

from torch.cuda import empty_cache

# Manually empty the CUDA cache before running
torch.cuda.empty_cache()

In [27]:
# Set environment variable before importing torch
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [9]:
# load image to test
from PIL import Image
image1 = Image.open('/content/drive/MyDrive/capstone_project/image_data/aluminum_can/Image_136.png')

In [43]:
torch.cuda.empty_cache()
instruction = 'What is this item in the image. Think about what is it made of and primarily used for, do not consider the brand, output a short answer in the format of: material item'

messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": instruction}
    ]}
]
input_text = processor.apply_chat_template(messages, add_generation_prompt=True)

inputs = processor(
    image1,
    input_text,
    add_special_tokens=False,
    return_tensors="pt"
).to(model.device)

#output = model.generate(**inputs, max_new_tokens=10)
#with autocast('cuda'):
with torch.amp.autocast('cuda'):
    output = model.generate(**inputs, max_new_tokens=20)

response_text = processor.decode(output[0]).split("<|start_header_id|>assistant<|end_header_id|>\n\n")[1].split("<|eot_id|>")[0].replace(".","")


OutOfMemoryError: CUDA out of memory. Tried to allocate 80.00 MiB. GPU 0 has a total capacity of 22.17 GiB of which 36.88 MiB is free. Process 2969 has 22.12 GiB memory in use. Of the allocated memory 21.56 GiB is allocated by PyTorch, and 341.65 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [13]:
print(response_text)

Aluminum can


In [39]:
knn_rag('aluminum can', 'Palm Desert', rag_embeddings, encoder_model)

(0,
 'The following metal and glass items can be recycled:\n         -Aluminum, steel, and tin cans,\n         -Clean aluminum foil,\n        - Glass bottles and jars (remove lids)',
 'aluminum can')

### Full 3-step Model

In [14]:
# RAG like backend

def retrive_instruct(class_choice, stage_1_output, location_name, recycling_instruction_dataset, locations_dict, instruc_mapping_df):

  locations_list = [x['location_id'] for x in recycling_instruction_dataset]

  loc_num = locations_dict[location_name]

  loc_idx = locations_list.index(loc_num)

  loc_dict = recycling_instruction_dataset[loc_idx]

  instructions_dict_list = loc_dict['instructions']


  relevant_sec_id = instruc_mapping_df[(instruc_mapping_df['pred_class']==class_choice) & (instruc_mapping_df['location_id']==loc_num)]['instruction_id'].values[0]
  stage_3_name =  instruc_mapping_df[(instruc_mapping_df['pred_class']==class_choice) & (instruc_mapping_df['location_id']==loc_num)]['generic names'].values[0]

  if stage_3_name == 'other':
    stage_3_name = stage_1_output

  instruc_id_list = [x['instruc_id'] for x in instructions_dict_list]

  instruc_idx = instruc_id_list.index(relevant_sec_id)

  retrieve_instruc = instructions_dict_list[instruc_idx]['instructions']

  return retrieve_instruc, stage_3_name

def knn_rag(stage_1_output, location, embeddings_cluster_dict, encoder_model):

  reference_words_list = list(embeddings_cluster_dict.keys())
  # encoder_model = SentenceTransformer('sentence-transformers/LaBSE')
  reference_embeddings = encoder_model.encode(reference_words_list)

  knn_model = NearestNeighbors(n_neighbors=1)
  knn_model.fit(reference_embeddings)

  query_phrase = stage_1_output.lower().strip(". ")

  query_llava = encoder_model.encode(query_phrase)

  dists, idxs = knn_model.kneighbors([query_llava])

  # print(idxs)
  neighbor = reference_words_list[idxs[0][0]]

  distance = dists[0]

  if distance <0.70:
    class_num = embeddings_cluster_dict[neighbor]
  else:
    class_num = 7

  instruction, stage_3_name = retrive_instruct(class_num, stage_1_output, location, recycling_instructions_open, locations_json_idx_dict, instruct_mapping_csv)

  return class_num, instruction, stage_3_name

In [15]:
def correct_instruction_retrieve(class_choice, location_name, locations_dict, recycling_instruction_dataset, instruc_mapping_df):
  locations_list = [x['location_id'] for x in recycling_instruction_dataset]

  loc_num = locations_dict[location_name]

  loc_idx = locations_list.index(loc_num)

  loc_dict = recycling_instruction_dataset[loc_idx]

  instructions_dict_list = loc_dict['instructions']


  relevant_sec_id = instruc_mapping_df[(instruc_mapping_df['pred_class']==class_choice) & (instruc_mapping_df['location_id']==loc_num)]['instruction_id'].values[0]

  instruc_id_list = [x['instruc_id'] for x in instructions_dict_list]

  instruc_idx = instruc_id_list.index(relevant_sec_id)

  retrieve_instruc = instructions_dict_list[instruc_idx]['instructions']

  return retrieve_instruc



In [17]:
def get_cv_response(img, instruction, num_new_token):

  image = img

  messages = [
      {"role": "user", "content": [
          {"type": "image"},
          {"type": "text", "text": instruction}
      ]}
  ]
  input_text = processor.apply_chat_template(messages, add_generation_prompt=True)

  inputs = processor(
      image,
      input_text,
      add_special_tokens=False,
      return_tensors="pt"
  ).to(model.device)

  with torch.amp.autocast('cuda'):
      output = model.generate(**inputs, max_new_tokens=20)

  response_text = processor.decode(output[0]).split("<|start_header_id|>assistant<|end_header_id|>\n\n")[1].split("<|eot_id|>")[0].replace(".","")

  return response_text

In [40]:
def run_stage_1(input_img, loc, processor, model, encoder_model):

  image = input_img

  messages = [
      {"role": "user", "content": [
          {"type": "image"},
          {"type": "text", "text": 'What is this item in the image. Think about what is it made of and primarily used for, do not consider the brand, output a short answer in the format of: material item'}
      ]}
  ]
  input_text = processor.apply_chat_template(messages, add_generation_prompt=True)

  inputs = processor(
      image,
      input_text,
      add_special_tokens=False,
      return_tensors="pt"
  ).to(model.device)

  with torch.amp.autocast('cuda'):
      output = model.generate(**inputs, max_new_tokens=20)

  response_text = processor.decode(output[0]).split("<|start_header_id|>assistant<|end_header_id|>\n\n")[1].split("<|eot_id|>")[0].replace(".","")
  return response_text
  #stage_1 = output_str[output_idx:]

  #class_num, instruction, stage_3_name = knn_rag(response_text, loc, rag_embeddings, encoder_model)

  #return class_num

In [35]:
knn_rag('aluminum can', 'Palm Desert', class_ref_dict, encoder_model)

(7,
 'This item is not included in the list of acceptable recyclables, please dispose of it in the garbage.',
 'plastic cup')

In [41]:
#encoder_model = SentenceTransformer('sentence-transformers/LaBSE')
run_stage_1(image1, 'Palm Desert', processor, model, encoder_model)

OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB. GPU 0 has a total capacity of 22.17 GiB of which 44.88 MiB is free. Process 2969 has 22.12 GiB memory in use. Of the allocated memory 21.61 GiB is allocated by PyTorch, and 289.51 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
def run_stage_2(input_img, loc, processor, model, stage_1, class_num):

  instruction = correct_instruction_retrieve(class_num, loc, locations_json_idx_dict, recycling_intsructions_open, instruct_mapping_csv)

  prompt = f"USER: <image>\n Given these instructions: {instruction}. Is {stage_1} in the image recyclable Choose yes or no\nASSISTANT:"

  inputs = processor(prompt, input_img, return_tensors='pt').to(0, torch.float16)

  output = model.generate(**inputs, max_new_tokens=5, do_sample=False, temperature = 0)

  output_str = processor.decode(output[0][2:], skip_special_tokens=True)

  output_idx = output_str.index("ASSISTANT") + 11
  stage_2 = output_str[output_idx:]

  return stage_2

In [ ]:
def run_stage_3(input_img, loc, processor, model, stage_1, stage_2):

  instruction = correct_instruction_retrieve(class_num, loc, locations_json_idx_dict, recycling_intsructions_open, instruct_mapping_csv)

  prompt = prompt = f"""USER: <image>\n Write recycling instructions based upon the following information about local recycling: {instruction}. Is this item able to be reyclced: {stage_2}. The descriptive name for the item is {stage_1}.
State in full sentences if this specific item can be recycled. Please provide instructions for how to recycle, create bulleted list instructions if it is recyclable, limit the answer to 3 sentences.\nASSISTANT:"""

  inputs = processor(prompt, input_img, return_tensors='pt').to(0, torch.float16)

  output = model.generate(**inputs, max_new_tokens=150, do_sample=False, temperature = 0)

  output_str = processor.decode(output[0][2:], skip_special_tokens=True)

  output_idx = output_str.index("ASSISTANT") + 11
  stage_3 = output_str[output_idx:]

  return stage_3

In [ ]:
# run the llava model on all 3 stages on the test set

locations_list = ['Palm Desert', 'Cook County']

# scoring_sentences_df = pd.DataFrame(columns=['image_id', 'location', 'true_label', 'pred_label', 'pulled_instructions', 'generated_sentence'])

stage_1_results_df = pd.DataFrame(columns = ['true', 'pred'])
stage_2_results_df = pd.DataFrame(columns = ['true', 'pred'])
stage_3_results_df = pd.DataFrame(columns = ['item', 'location', 'outputted sentence'])


for i in range(len(y_data)):
  img = Image.fromarray(np.uint8(x_data[i]))

  class_num = class_ref_dict[y_data[i]]

  loc_choice = random.randint(0, 1)
  loc_name = locations_list[loc_choice]

  # locations_list = [x['location_id'] for x in recycling_instructions_open]

  loc_num = locations_json_idx_dict[loc_name]

  s2_true = test_s2_answers_df[(test_s2_answers_df['class']==class_num) & (test_s2_answers_df['location_id']==loc_num)]['recyclability'].values[0]

  generic_name = instruct_mapping_csv[(instruct_mapping_csv['pred_class']==class_num) & (instruct_mapping_csv['location_id']==loc_num)]['generic names'].values[0]

  s1_pred = run_stage_1(img, loc_name, processor, model, encoder_model)

  s2_pred = run_stage_2(img, loc_name, processor, model, generic_name, class_num)

  s3_pred = run_stage_3(img, loc_name, processor, model, generic_name, s2_true)


  # pred_name, relevant_instructions, model_output = run_model(img, loc_name, processor, model, encoder_model)

  # save results
  stage_1_results_df.loc[len(stage_1_results_df.index)] = [class_num, s1_pred]
  stage_2_results_df.loc[len(stage_2_results_df.index)] = [s2_true, s2_pred]
  stage_3_results_df.loc[len(stage_3_results_df.index)] = [generic_name, loc_name, s3_pred]

stage_1_results_df.to_csv('/content/drive/My Drive/capstone_personal/results/test_data_s1_results.csv')
stage_2_results_df.to_csv('/content/drive/My Drive/capstone_personal/results/test_data_s2_results.csv')
stage_3_results_df.to_csv('/content/drive/My Drive/capstone_personal/results/test_data_s3_results.csv')
# scoring_sentences_df

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_util